0.0 Imports

In [48]:
import pandas as pd

0.1 Helper Functioons

0.2 Loading data

In [63]:
df_sales_raw = pd.read_csv('./data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('./data/store.csv', low_memory=False)

#df_raw = pd.concat([df_sales_raw, df_store_raw]) - juntando dois datasets

df_raw = pd.merge(df_sales_raw, df_store_raw, how = 'left', on = 'Store')

In [60]:
df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
651667,178,7.0,2013-11-24,0.0,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1.0 Descrição dos dados

In [66]:
df1 = df_raw.copy()

1.1 Rename Columns

In [64]:
df_raw.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')